# Problem Description

This problem involves using Recurrent neural networks and natural language processing to classify sequential text data, namely tweets that may or may not be reporting on a natural distaster which has occurred. the training data consists of a list of tweets and labels on whether or not the tweet contains information pertaining to a disaster. Due to the structure of this data, a bag of words approach will not be as effective, as some tweets may contain words that are used in disaster tweets (e.g. "fire") that are used in other contexts. our task is to create a recurrent neural network that will accurately classify these tweets, which will then be used to identify unlabelled tweets, which will be submitted to the Kaggle competition.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import string

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

# EDA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [ ]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## Data Cleaning

We will be removing stopwords (common words such as the) and converting all text data to words by removing all punctuation and converting the remaining words to lowercase. this will allow us to convert individual words to tokens that can be fed into our recurrent neural network with minimal confounds

In [ ]:
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))

def clean_str(s):
    # strip punctuation
    s = s.translate(str.maketrans('', '', string.punctuation))
    # lowercase
    s = s.lower()
    # remove stopwords
    lists = [word for word in s.split() if word not in stop]
    return ' '.join(lists)

train['text'] = train['text'].astype(str)
train['text'] = train['text'].map(lambda s: clean_str(s))
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,residents asked shelter place notified officer...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1


## Preprocessing

### Tokenizing

We now convert our text data that has been stripped of punctuation and capitalization and common words to a list of values using the Keras Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features=5000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(train['text'].values)
X = tokenizer.texts_to_sequences(train['text'].values)
X = pad_sequences(X)
y = train['target']
X

array([[   0,    0,    0, ..., 1454, 4391,   13],
       [   0,    0,    0, ...,  134,  575, 1237],
       [   0,    0,    0, ...,  547, 1238,  950],
       ...,
       [   0,    0,    0, ..., 4382,  482, 1443],
       [   0,    0,    0, ..., 1074, 2447,  203],
       [   0,    0,    0, ...,   79,  596,   10]], dtype=int32)

### Embedding - GLoVe

In order to train our neural network, we can save time on training the network by using a set of weights that have been trained on a much larger corpus of text data. For this purpose, we will use the GloVe data downloaded from Stanford.

In [ ]:
# Source: https://www.kaggle.com/code/mariapushkareva/nlp-disaster-tweets-with-glove-and-lstm

embeddings_dictionary = dict()
embedding_dim = 100
glove = open('/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt')
for line in glove:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove.close()

In [ ]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
embedding_matrix.shape

(22565, 100)

# Modeling

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =42)

In [ ]:
from keras import models, layers

## Base Model - LTSM

For our model, we will use LTSM, which stands for Long Short-term memory. This is a recurrent neural network that stores a context window that varies in length based on how relevant the surrounding information is. Our model  

In [ ]:
lstm_model = models.Sequential([
    layers.Embedding(input_dim=22565,
        output_dim=100,
        weights = [embedding_matrix]),
    layers.Bidirectional(layers.LSTM(
        100,
        return_sequences = True,
        recurrent_dropout=0.2)),
    layers.GlobalMaxPool1D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(100, activation = "relu"),
    layers.Dropout(0.5),
    layers.Dense(100, activation = "relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation = 'sigmoid'),


])

lstm_model.compile(optimizer='rmsprop',
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

lstm_model.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)             │ ?                           │       2,256,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_10              │ ?                           │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_30 (Dropout)                 │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_32 (Dropout)                 │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,256,500 (8.61 MB)

 Trainable params: 2,256,500 (8.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_model.fit(X_train, y_train, epochs = 10, validation_data=(X_test, y_test))

Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 29s 107ms/step - accuracy: 0.6325 - loss: 0.7896 - val_accuracy: 0.8004 - val_loss: 0.5536
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 40s 102ms/step - accuracy: 0.7551 - loss: 0.5380 - val_accuracy: 0.8050 - val_loss: 0.4743
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 112ms/step - accuracy: 0.7830 - loss: 0.4718 - val_accuracy: 0.7984 - val_loss: 0.4662
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 41s 113ms/step - accuracy: 0.8146 - loss: 0.4438 - val_accuracy: 0.8076 - val_loss: 0.4377
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 0.8309 - loss: 0.4093 - val_accuracy: 0.8030 - val_loss: 0.4511
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 111ms/step - accuracy: 0.8339 - loss: 0.3952 - val_accuracy: 0.8148 - val_loss: 0.4559
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 42s 116ms/step - accuracy: 0.8487 - loss: 0.3746 - val_accuracy: 0.8148 - val_loss: 0.4677
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 20s 103ms/step - accuracy: 0.8515 - loss: 0

# Results and Analysis

## Comparison of Different Architectures

### GRU

Here we compare our LTSM model to another model architecture, GRU. This model appears to overfit more quickly than LTSM and does not perform noticeably better.

In [ ]:
gru_model = models.Sequential([
    layers.Embedding(input_dim=22565,
        output_dim=100,
        weights = [embedding_matrix]),
    layers.Bidirectional(layers.GRU(
        100,
        return_sequences = True,
        recurrent_dropout=0.2)),
    layers.GlobalMaxPool1D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(100, activation = "relu"),
    layers.Dropout(0.5),
    layers.Dense(100, activation = "relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation = 'sigmoid'),


])

gru_model.compile(optimizer='rmsprop',
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

gru_model.fit(X_train, y_train, epochs = 10, validation_data=(X_test, y_test))

gru_model.summary()

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


191/191 ━━━━━━━━━━━━━━━━━━━━ 29s 108ms/step - accuracy: 0.6351 - loss: 0.7432 - val_accuracy: 0.7761 - val_loss: 0.5608
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - accuracy: 0.7529 - loss: 0.5319 - val_accuracy: 0.8168 - val_loss: 0.4767
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 24s 123ms/step - accuracy: 0.7923 - loss: 0.4679 - val_accuracy: 0.8181 - val_loss: 0.4289
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 38s 106ms/step - accuracy: 0.8210 - loss: 0.4263 - val_accuracy: 0.8050 - val_loss: 0.4599
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 20s 104ms/step - accuracy: 0.8300 - loss: 0.3997 - val_accuracy: 0.8181 - val_loss: 0.4410
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 21s 106ms/step - accuracy: 0.8345 - loss: 0.3998 - val_accuracy: 0.8155 - val_loss: 0.4572
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 20s 101ms/step - accuracy: 0.8462 - loss: 0.3634 - val_accuracy: 0.8102 - val_loss: 0.4662
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 22s 109ms/step - accuracy: 0.8630 - loss: 0.3404 - val

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_14 (Embedding)             │ (None, 21, 100)             │       2,256,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ (None, 21, 200)             │         121,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_11              │ (None, 200)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 200)                 │             800 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_33 (Dropout)                 │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 100)                 │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_34 (Dropout)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 100)                 │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_35 (Dropout)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,817,204 (18.38 MB)

 Trainable params: 2,408,401 (9.19 MB)

 Non-trainable params: 400 (1.56 KB)

 Optimizer params: 2,408,403 (9.19 MB)

## Performance

## Hyperparameter Tuning

The hyperparameters for the LTSM model, which appears to perform better than GRU, can now be tuned using the keras tuner package.

In [ ]:
!pip install keras_tuner
import keras_tuner as kt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
def model_builder(hp):
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-96
    hp_dropout = hp.Float('dropout', min_value=.1, max_value=.5, step=.1)

    model = models.Sequential([
    layers.Embedding(input_dim=22565,
        output_dim=100,
        weights = [embedding_matrix]),
    layers.Bidirectional(layers.LSTM(
        100,
        return_sequences = True,
        recurrent_dropout=hp_dropout)),
    layers.GlobalMaxPool1D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(100, activation = "relu"),
    layers.Dropout(0.5),
    layers.Dense(100, activation = "relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation = 'sigmoid'),
    ])

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])


    return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=30,
                     factor=3)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(X_train, y_train, epochs = 10, validation_data=(X_test, y_test), callbacks=[stop_early])

Trial 15 Complete [00h 01m 03s]
val_accuracy: 0.7892317771911621

Best val_accuracy So Far: 0.8128693103790283
Total elapsed time: 00h 19m 38s


In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete. The optimal number of units in the dropout
layer is {best_hps.get('dropout')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")
tuner.search_space_summary(extended=True)


The hyperparameter search is complete. The optimal number of units in the dropout
layer is 0.5 and the optimal learning rate for the optimizer
is 0.001.

Search space summary
Default search space size: 2
dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


## Best Model

After tuning out dropout and learning rates for LTSM, we can train our best model on the training data in order to process the test data

In [ ]:
best_model = models.Sequential([
    layers.Embedding(input_dim=22565,
        output_dim=100,
        weights = [embedding_matrix]),
    layers.Bidirectional(layers.LSTM(
        100,
        return_sequences = True,
        recurrent_dropout=0.2)),
    layers.GlobalMaxPool1D(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(100, activation = "relu"),
    layers.Dropout(0.5),
    layers.Dense(100, activation = "relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation = 'sigmoid'),


])

best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001),
                       loss='binary_crossentropy',
                       metrics=['accuracy'])


best_model.fit(X_train, y_train, epochs = 5, validation_data=(X_test, y_test))

Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 37s 123ms/step - accuracy: 0.6479 - loss: 0.7337 - val_accuracy: 0.7978 - val_loss: 0.5861
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 41s 121ms/step - accuracy: 0.7514 - loss: 0.5260 - val_accuracy: 0.8142 - val_loss: 0.4725
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 41s 122ms/step - accuracy: 0.8055 - loss: 0.4476 - val_accuracy: 0.8122 - val_loss: 0.4399
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 40s 117ms/step - accuracy: 0.8321 - loss: 0.4007 - val_accuracy: 0.8142 - val_loss: 0.4547
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 21s 108ms/step - accuracy: 0.8375 - loss: 0.3989 - val_accuracy: 0.8188 - val_loss: 0.4370


## Submission

In [ ]:
test['text'] = test['text'].astype(str)
test['text'] = test['text'].map(lambda s: clean_str(s))
test_data = tokenizer.texts_to_sequences(test['text'].values)
test_data = pad_sequences(test_data)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
submission.target = best_model.predict(test_data)
submission.target = submission.target.round(0)
submission.to_csv("/content/drive/MyDrive/Colab Notebooks/submission.csv", index=False)

102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


# Conclusion

While the two model architectures, LTSM and GRU, largely performed similarly, and the hyperparamter tuning showed only minimal improvement or deviation, it seems that these types of models can consistently predict this sort of natural language data with approximately 80% accuracy.

# References
1. https://www.kaggle.com/code/alexia/kerasnlp-starter-notebook-disaster-tweets
2. https://keras.io/examples/nlp/masked_language_modeling/
3. https://www.kaggle.com/code/tuckerarrants/disaster-tweets-eda-glove-rnns-bert#2.-Simple-LSTM-Model
4. https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
5. https://www.kaggle.com/code/andreshg/nlp-glove-bert-tf-idf-lstm-explained#7.-LSTM
6. https://towardsdatascience.com/mamba-ssm-theory-and-implementation-in-keras-and-tensorflow-32d6d4b32546/